# Churn Prediction API - Test Notebook

This notebook demonstrates how to interact with the Churn Prediction API.

## Prerequisites

Make sure the API server is running:
```bash
cd "c:\Users\samar\Desktop\Projects\Notebook Deploy"
python src/main.py
```

The server should be running at: `http://127.0.0.1:8000`

In [ ]:
import requests
import pandas as pd
import json
from pprint import pprint

## Configuration

In [ ]:
# API Base URL
BASE_URL = "http://127.0.0.1:8000"
API_V1 = f"{BASE_URL}/api/v1"

print(f"API Base URL: {BASE_URL}")
print(f"API v1 Endpoint: {API_V1}")

## Test 1: Health Check

Verify the API is running and healthy.

In [ ]:
response = requests.get(f"{BASE_URL}/health")
print(f"Status Code: {response.status_code}")
print("Response:")
pprint(response.json())

## Test 2: Load Sample Data

Load the sample customer data we'll use for predictions.

In [ ]:
# Load sample data
df = pd.read_csv('src/data/input/sample_customers.csv')
print(f"Loaded {len(df)} customer records")
df.head()

## Test 3: Make Predictions

Send a prediction request to the API.

In [ ]:
# Convert first 5 customers to list of dictionaries
sample_data = df.head(5).to_dict('records')

# Create prediction request
payload = {
    "data": sample_data
}

print("Sending prediction request...")
print(f"Number of records: {len(sample_data)}")

In [ ]:
# Make the API call
response = requests.post(
    f"{API_V1}/predict",
    json=payload,
    headers={"Content-Type": "application/json"}
)

print(f"Status Code: {response.status_code}")

if response.status_code == 200:
    result = response.json()
    print("\n=== Prediction Results ===")
    print(f"Model Version: {result['model_version']}")
    print(f"Execution Time: {result['execution_time']:.2f}s")
    print(f"Notebook Path: {result['notebook_path']}")
    print(f"\nPredictions (Churn Probability):")
    for i, prob in enumerate(result['predictions']):
        print(f"  Customer {i+1}: {prob:.3f}")
else:
    print("Error:")
    print(response.text)

## Test 4: Visualize Results

Create a simple visualization of the predictions.

In [ ]:
import matplotlib.pyplot as plt

if response.status_code == 200:
    result = response.json()
    predictions = result['predictions']
    
    # Create visualization
    plt.figure(figsize=(10, 6))
    plt.bar(range(len(predictions)), predictions, color='steelblue', alpha=0.7)
    plt.axhline(y=0.5, color='red', linestyle='--', label='50% Threshold')
    plt.xlabel('Customer Index')
    plt.ylabel('Churn Probability')
    plt.title('Customer Churn Predictions')
    plt.ylim(0, 1)
    plt.legend()
    plt.grid(axis='y', alpha=0.3)
    plt.tight_layout()
    plt.show()
    
    # Risk categorization
    high_risk = sum(1 for p in predictions if p > 0.7)
    medium_risk = sum(1 for p in predictions if 0.4 <= p <= 0.7)
    low_risk = sum(1 for p in predictions if p < 0.4)
    
    print("\n=== Risk Distribution ===")
    print(f"High Risk (>70%): {high_risk} customers")
    print(f"Medium Risk (40-70%): {medium_risk} customers")
    print(f"Low Risk (<40%): {low_risk} customers")

## Test 5: Batch Prediction

Test with all sample customers.

In [ ]:
# Send all customers for prediction
all_data = df.to_dict('records')

payload_all = {
    "data": all_data
}

print(f"Sending batch prediction request for {len(all_data)} customers...")

response_all = requests.post(
    f"{API_V1}/predict",
    json=payload_all,
    headers={"Content-Type": "application/json"}
)

if response_all.status_code == 200:
    result_all = response_all.json()
    print(f"\n✓ Successfully predicted {len(result_all['predictions'])} customers")
    print(f"Execution Time: {result_all['execution_time']:.2f}s")
    print(f"Average Churn Probability: {sum(result_all['predictions'])/len(result_all['predictions']):.3f}")
else:
    print(f"Error: {response_all.status_code}")
    print(response_all.text)

## Test 6: Check Drift Report

Get the latest drift detection report (if available).

In [ ]:
response_drift = requests.get(f"{API_V1}/drift-report")

print(f"Status Code: {response_drift.status_code}")

if response_drift.status_code == 200:
    drift_report = response_drift.json()
    print("\n=== Drift Report ===")
    pprint(drift_report)
else:
    print("Note: Drift report may not be available yet (this is normal for first run)")
    print(response_drift.text)

## Test 7: API Documentation

The API provides interactive documentation via Swagger UI.

**Access it at:** [http://127.0.0.1:8000/docs](http://127.0.0.1:8000/docs)

You can also view the OpenAPI schema at: [http://127.0.0.1:8000/openapi.json](http://127.0.0.1:8000/openapi.json)

In [ ]:
# Fetch OpenAPI schema
response_schema = requests.get(f"{BASE_URL}/openapi.json")

if response_schema.status_code == 200:
    schema = response_schema.json()
    print("Available Endpoints:")
    for path, methods in schema['paths'].items():
        for method in methods.keys():
            print(f"  {method.upper()} {path}")
else:
    print("Could not fetch API schema")

## Summary

This notebook demonstrated:
1. ✓ Health check endpoint
2. ✓ Loading sample data
3. ✓ Making predictions via API
4. ✓ Visualizing prediction results
5. ✓ Batch predictions
6. ✓ Drift monitoring endpoint
7. ✓ API documentation access

### Next Steps

- Explore the Swagger UI at http://127.0.0.1:8000/docs
- Check executed notebooks in `src/notebooks/executed/`
- Review prediction outputs in `src/data/output/`
- Monitor metrics at http://127.0.0.1:8000/metrics (Prometheus format)